<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 3.2.1 
# *Querying the International Space Station*

## The OpenNotify API

The OpenNotify API exposes a few attributes of the International Space Station (ISS) via a simple, authentication-free interface. The simplicity of this API precludes any need for a dedicated Python library. However, as with many APIs, it accepts requests according to HTTP standards and returns responses in JSON format, so the Python libraries request and json will make managing the I/O simpler still.

In [1]:
import requests
import json
from datetime import datetime, date, time

This request fetches the latest position of the international space station:

In [2]:
response = requests.get("http://api.open-notify.org/iss-now.json")

Print the status code of the response:

In [3]:
print(response.status_code)

200


We can also request the current position of the ISS and the next few times at which it will be over a certain location. The latitude and longitude of Sydney are (-33.87, 151.21). Create a dict named `parameters` whose elements are key:value pairs named `lat` and `lon`, containing those figures:

In [4]:
parameters = {'lat':-33.87, 'lon':151.21}

The API request we want starts like the last one, but the last part of the endpoint is "iss-pass.json", and it takes a second argument -- namely, the `parameters` value created above. Compose and execute the API request:

In [6]:
response_sydney = requests.get("http://api.open-notify.org/iss-pass.json", parameters)

Print the response header:

In [9]:
print(response_sydney.headers)

{'Server': 'nginx/1.10.3', 'Date': 'Thu, 27 Jan 2022 07:27:40 GMT', 'Content-Type': 'application/json', 'Content-Length': '521', 'Connection': 'keep-alive', 'Via': '1.1 vegur'}


Print the content of the response (the data that the server returned):

In [10]:
print(response_sydney.content)

b'{\n  "message": "success", \n  "request": {\n    "altitude": 100, \n    "datetime": 1643268460, \n    "latitude": -33.87, \n    "longitude": 151.21, \n    "passes": 5\n  }, \n  "response": [\n    {\n      "duration": 614, \n      "risetime": 1643297363\n    }, \n    {\n      "duration": 638, \n      "risetime": 1643303149\n    }, \n    {\n      "duration": 463, \n      "risetime": 1643309098\n    }, \n    {\n      "duration": 349, \n      "risetime": 1643315063\n    }, \n    {\n      "duration": 522, \n      "risetime": 1643320876\n    }\n  ]\n}\n'


Note that this is a Python byte string:

In [12]:
print(response_sydney.content.decode())

{
  "message": "success", 
  "request": {
    "altitude": 100, 
    "datetime": 1643268460, 
    "latitude": -33.87, 
    "longitude": 151.21, 
    "passes": 5
  }, 
  "response": [
    {
      "duration": 614, 
      "risetime": 1643297363
    }, 
    {
      "duration": 638, 
      "risetime": 1643303149
    }, 
    {
      "duration": 463, 
      "risetime": 1643309098
    }, 
    {
      "duration": 349, 
      "risetime": 1643315063
    }, 
    {
      "duration": 522, 
      "risetime": 1643320876
    }
  ]
}



Print just the "content-type" value from the header:

In [14]:
print(response_sydney.headers.get('Content-Type'))

application/json


JSON was designed to be easy for computers to read, not for people. The `requests` library can decode the JSON byte string:

In [15]:
print(response_sydney.json())

{'message': 'success', 'request': {'altitude': 100, 'datetime': 1643268460, 'latitude': -33.87, 'longitude': 151.21, 'passes': 5}, 'response': [{'duration': 614, 'risetime': 1643297363}, {'duration': 638, 'risetime': 1643303149}, {'duration': 463, 'risetime': 1643309098}, {'duration': 349, 'risetime': 1643315063}, {'duration': 522, 'risetime': 1643320876}]}


What kind of object did this give us?

In [16]:
print(type(response_sydney.json()))

<class 'dict'>


Python dicts are easier to work with, but the data we want is still buried in that data structure, so we have to dig it out. First, extract the `response` value to a separate dict:

In [17]:
response_dict = response_sydney.json()

Now extract the `risetime` strings into an array called `srisetimes`:

In [26]:
# Import Numpy library
import numpy as np

# Extract response list from request
responselist = response_dict.get('response')

# Extract 'risetime' strings into array using list comprehension
srisetimes = [i['risetime'] for i in responselist]

print(srisetimes)

[1643297363, 1643303149, 1643309098, 1643315063, 1643320876]


```
datetime.fromtimestamp(str)
```

Convert these to an array of Python `datetime` values called `risetimes`:

In [28]:
# Create simple function to convert to datetime
def converttotime(x):
    return datetime.fromtimestamp(x)

# Use map method to apply elementwise to list
risetimes = list(map(converttotime, srisetimes))

print(risetimes)

[datetime.datetime(2022, 1, 28, 2, 29, 23), datetime.datetime(2022, 1, 28, 4, 5, 49), datetime.datetime(2022, 1, 28, 5, 44, 58), datetime.datetime(2022, 1, 28, 7, 24, 23), datetime.datetime(2022, 1, 28, 9, 1, 16)]


Finally, print these in a format that people understand:



```
str.strftime('%d/%m/%y %I:%M')

27/03/19 10:59
27/03/19 12:35
27/03/19 02:11
28/03/19 03:42
28/03/19 05:13
```



In [30]:
for i in risetimes:
    print(i.strftime('%d/%m/%y %I:%M'))

28/01/22 02:29
28/01/22 04:05
28/01/22 05:44
28/01/22 07:24
28/01/22 09:01


Here is an endpoint that tells us who is onboard:

In [31]:
response = requests.get("http://api.open-notify.org/astros.json")

Referring to the methods used above, extract the number of astronauts and their names:

In [33]:
# First, check the contents. Assume data type of json
print(response.json())

{'people': [{'craft': 'ISS', 'name': 'Mark Vande Hei'}, {'craft': 'ISS', 'name': 'Pyotr Dubrov'}, {'craft': 'ISS', 'name': 'Anton Shkaplerov'}, {'craft': 'Shenzhou 13', 'name': 'Zhai Zhigang'}, {'craft': 'Shenzhou 13', 'name': 'Wang Yaping'}, {'craft': 'Shenzhou 13', 'name': 'Ye Guangfu'}, {'craft': 'ISS', 'name': 'Raja Chari'}, {'craft': 'ISS', 'name': 'Tom Marshburn'}, {'craft': 'ISS', 'name': 'Kayla Barron'}, {'craft': 'ISS', 'name': 'Matthias Maurer'}], 'message': 'success', 'number': 10}


In [35]:
# Result is a dictionary with a list of dictionaries as a value of one of the key:value pairs
# Pass dictionary to object
astrodict = response.json()

# From above, astronaut number is equal to the value corresponding to the 
#'number' key
no_astronauts =astrodict.get('number')

print(no_astronauts)

10


In [36]:
# Astronaut names are contained within list of dictionaries in the 'people' pair
# Extract using list comprehension of 'people' value
astronames = [i['name'] for i in astrodict.get('people')]

print(astronames)

['Mark Vande Hei', 'Pyotr Dubrov', 'Anton Shkaplerov', 'Zhai Zhigang', 'Wang Yaping', 'Ye Guangfu', 'Raja Chari', 'Tom Marshburn', 'Kayla Barron', 'Matthias Maurer']


## HOMEWORK


1. Write a simple handler for the response status code (refer to lab resources slide for HTTP response codes). As this Jupyter Notebook is an interactive device, the handler does not need to manage subsequent code execution (i.e. by branching or aborting execution), although it should return something that could be used to do so if deployed in a Python program.

In [49]:
#ANSWER:
def handleResponse(response, verbose = False):
    '''
    Returns Boolean Value, Status Code, 
    '''
    # if Status Code is 200 return false, and status code
    if verbose:
      Bool = True
      status_code = response.status_code
    else:
      if response.status_code == 200:
        Bool = False
        status_code = response.status_code
    # Otherwise Return True and Status Code
      else:
        Bool = True
        status_code = response.status_code

    return Bool, status_code

2. Test your response handler on some correct and incorrect API calls.

In [50]:
response = requests.get("http://api.open-notify.org/astrsaos.json")
if handleResponse(response)[0]:
    print('API call failed. Resolve issue before continuing!')
    
response = requests.get("http://api.open-notify.org/iss-pass.json")
handleResponse(response, True)[0]

API call failed. Resolve issue before continuing!


True

>

>

>



---



---



> > > > > > > > > © 2021 Institute of Data


---



---



